In [69]:
import pandas as pd
import os
folder_path = "/Users/rohinpalsule/Documents/GitHub/GraphWalkFlights/GWF_text_validation/data/new_batch"
df_concat = []
i = 1
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file)
        dfindividual = pd.read_csv(file_path,on_bad_lines='skip')  # Read the CSV
        dfindividual['partid']= dfindividual['responses'].str.replace('{worker_id:',"").str.replace('}', '')
        dfindividual['partid'] = dfindividual['partid'].ffill()
        dfindividual['age'] = 'young'
        df_concat.append(dfindividual)
        i +=1
df = pd.concat(df_concat,ignore_index=True)

In [70]:
df

,rt,responses,trial_type,time_elapsed,stimulus,button_pressed,key_press,name_trial,rating,probe,...,familiar,memorable,unique,realistic,similar,comments,detectfocus,warning,partid,age
0,14491.0,{worker_id:5c8f2ac8d7ecab0018d529aa},id_enter,14493,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5c8f2ac8d7ecab0018d529aa,young
1,10169.0,NaN,fullscreen,24667,fullscreen,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5c8f2ac8d7ecab0018d529aa,young
2,6452.0,NaN,intro_0,31122,instruct,NaN,32.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5c8f2ac8d7ecab0018d529aa,young
3,13366.0,NaN,intro_1,44489,instruct,NaN,32.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5c8f2ac8d7ecab0018d529aa,young
4,6001.0,NaN,intro_2,50490,instruct,NaN,32.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5c8f2ac8d7ecab0018d529aa,young
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17382,5754.8,NaN,recognition_phase,1296981,Frankfort,NaN,49.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6705f9372b81e6f4e1e195df,young
17383,NaN,NaN,confidence_buffer,1297083,confidence_buffer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6705f9372b81e6f4e1e195df,young
17384,NaN,NaN,thebreak,1297584,break,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6705f9372b81e6f4e1e195df,young
17385,NaN,NaN,survey,1504333,survey-questions,NaN,NaN,NaN,NaN,NaN,...,If I have heard it at all in conversations in ...,if it was unique enough to stand out,if its a city name that I dont think every sta...,mainly whether or not I had heard it before an...,no,maybe not as many cities as it did get very re...,NaN,NaN,6705f9372b81e6f4e1e195df,young


In [71]:
# Data Pivot
ratings_df = df[df['trial_type'].str.endswith('rating', na=False)]
df_wide = ratings_df.pivot_table(
    index=['partid', 'stimulus'], 
    columns='trial_type', 
    values='rating',
    aggfunc='first'  # use first if duplicates exist
).reset_index()

# Flatten
df_wide.columns.name = None
df_wide.columns = [str(col) for col in df_wide.columns]

# Column order
desired_order = [
    'partid', 'stimulus',
    'familiar_rating', 'unique_rating', 'memorable_rating','realistic_rating',
    'pt2_unique_rating', 'pt2_memorable_rating'
]
df_wide = df_wide[[col for col in desired_order if col in df_wide.columns]]
df_wide = df_wide.drop(columns='partid')
# participant | city | familiar_rating | unique_rating | memorable_rating | realistic_rating | pt2_unique_rating | pt2_memorable_rating
city_df = df_wide.groupby('stimulus').agg(['mean','sem']).reset_index()
city_mean_df = df_wide.groupby('stimulus').mean().reset_index()


In [72]:
city_mean_df.sort_values('familiar_rating',ascending=False).reset_index(drop='index')

,stimulus,familiar_rating,unique_rating,memorable_rating,realistic_rating,pt2_unique_rating,pt2_memorable_rating
0,Santa Monica,4.714286,3.142857,4.571429,4.666667,2.958333,3.708333
1,Athens,4.642857,3.500000,4.500000,4.571429,3.047619,3.952381
2,Tallahassee,4.500000,3.642857,4.000000,4.480000,3.800000,4.080000
3,Cambridge,4.454545,2.727273,4.363636,4.458333,3.000000,3.583333
4,Pasadena,4.230769,3.230769,4.153846,4.480000,3.600000,3.960000
5,Concord,4.000000,2.700000,2.800000,4.260870,2.913043,3.260870
6,Reno,3.857143,3.714286,4.000000,4.440000,3.280000,3.600000
7,Boise,3.857143,3.571429,3.428571,4.240000,3.400000,3.360000
8,Syracuse,3.846154,3.923077,3.538462,4.541667,3.750000,3.875000
9,Salem,3.833333,3.166667,3.500000,4.708333,2.916667,3.750000


In [73]:
city_df

stimulus familiar_rating           unique_rating            \
                            mean       sem          mean       sem   
0        Abilene        2.785714  0.446950      3.285714  0.369564   
1        Altoona        1.727273  0.332782      3.818182  0.443502   
2      Ann Arbor        3.500000  0.452267      3.500000  0.379393   
3      Annapolis        2.500000  0.374313      3.142857  0.375882   
4         Athens        4.642857  0.225007      3.500000  0.374313   
5     Bellingham        2.545455  0.434085      3.363636  0.278722   
6       Berkeley        3.733333  0.383799      3.133333  0.273716   
7          Boise        3.857143  0.553283      3.571429  0.480929   
8        Boulder        3.700000  0.422953      2.200000  0.442217   
9        Bozeman        2.428571  0.611678      3.285714  0.565445   
10    Burlington        3.090909  0.314918      2.454545  0.412611   
11     Cambridge        4.454545  0.312283      2.727273  0.304240   
12   Carson City        2.500000  0.477516      2.428571  0.326727   
13     Champaign        2.230769  0.410657      2.384615  0.310881   
14      Cheyenne        2.692308  0.382043      3.153846  0.372898   
15       Concord        4.000000  0.471405      2.700000  0.422953   
16       Decatur        3.111111  0.538631      3.777778  0.364302   
17        Dothan        1.916667  0.378561      3.083333  0.483960   
18         Dover        3.300000  0.538516      2.800000  0.489898   
19       Dubuque        1.846154  0.355292      3.307692  0.472101   
20         Fargo        3.083333  0.451569      2.750000  0.371830   
21  Fort Collins        2.333333  0.309773      2.416667  0.336162   
22    Fort Wayne        3.562500  0.364792      2.187500  0.291815   
23     Frankfort        2.625000  0.497763      2.125000  0.479490   
24   Gainesville        3.222222  0.464811      2.333333  0.408248   
25    Hagerstown        1.642857  0.386690      3.571429  0.358787   
26        Helena        2.882353  0.382353      2.470588  0.298497   
27        Ithaca        2.500000  0.534522      3.875000  0.350382   
28    Las Cruces        2.000000  0.404520      3.909091  0.314918   
29     Marquette        2.066667  0.330464      3.466667  0.236375   
30      McKinney        2.400000  0.452155      2.500000  0.428174   
31    Montpelier        2.416667  0.451569      4.083333  0.287580   
32        Muncie        1.222222  0.146986      3.888889  0.351364   
33     New Haven        3.214286  0.421647      2.285714  0.369564   
34  Newport News        2.923077  0.473145      3.230769  0.360801   
35       Norfolk        3.714286  0.384144      3.000000  0.363137   
36       Olympia        3.636364  0.452724      3.090909  0.435985   
37     Palo Alto        3.076923  0.329365      3.923077  0.309291   
38      Pasadena        4.230769  0.302846      3.230769  0.302846   
39        Peoria        2.000000  0.381385      3.636364  0.363636   
40        Pueblo        2.200000  0.359011      3.000000  0.298142   
41          Reno        3.857143  0.553283      3.714286  0.420560   
42  Rock Springs        2.142857  0.442981      2.285714  0.304338   
43         Salem        3.833333  0.365839      3.166667  0.321769   
44  Santa Monica        4.714286  0.194105      3.142857  0.345412   
45    Scottsdale        3.764706  0.303526      2.529412  0.272753   
46      Stamford        2.714286  0.424893      2.571429  0.402114   
47      Syracuse        3.846154  0.317162      3.923077  0.264612   
48   Tallahassee        4.500000  0.251370      3.642857  0.357143   
49   Terre Haute        2.083333  0.434468      4.000000  0.408248   
50        Toledo        3.733333  0.407859      3.266667  0.330464   
51        Topeka        3.000000  0.452911      3.846154  0.249259   
52        Yakima        3.272727  0.573520      4.181818  0.377026   
53    Youngstown        2.727273  0.428335      2.636364  0.432177   

   memorable_rating           realistic_rating           pt2_unique_rating  \
         

In [75]:
survey_df = df[df['trial_type']=='survey']
survey_df[['partid','smooth', 'distraction', 'strategies', 'familiar', 'memorable',
       'unique', 'realistic', 'similar', 'comments']].to_csv("survey_responses.csv",index=False)